**Индивидуальное задание**

1. Набор данных winequality-red.csv.
2. Постройте модель многомерной регрессии с использованием стратегии backward elimination.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

data = pd.read_csv("winequality-white.csv", sep=';')

# Разделение на признаки и целевую переменную
X = data.drop(columns=["quality"])  # Все признаки, кроме качества
y = data["quality"]  # Целевая переменная

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Функция для вычисления коэффициентов и p-value
def calculate_p_values(X, y, model):
    n = X.shape[0]
    p = X.shape[1]
    model.fit(X, y)
    predictions = model.predict(X)
    
    residuals = y - predictions
    mse = np.sum(residuals**2) / (n - p)  # Mean squared error
    
    # Стандартные ошибки для коэффициентов
    XtX_inv = np.linalg.inv(np.dot(X.T, X))
    std_errors = np.sqrt(np.diagonal(XtX_inv) * mse)
    
    # p-value для каждого коэффициента
    p_values = 2 * (1 - norm.cdf(np.abs(model.coef_ / std_errors)))
    
    return p_values

# Создание линейной модели
model = LinearRegression()

# Изначальная модель с использованием всех признаков
p_values = calculate_p_values(X_train_scaled, y_train, model)

# Backward elimination
features = X.columns.tolist()
while np.max(p_values) > 0.05:
    max_p_value_index = np.argmax(p_values)
    excluded_feature = features[max_p_value_index]
    features.remove(excluded_feature)
    
    # Пересоздаем выборку X_train и X_test без исключенного признака
    X_train_reduced = X_train[features]
    X_test_reduced = X_test[features]
    
    # Масштабируем данные без исключенного признака
    X_train_reduced_scaled = scaler.fit_transform(X_train_reduced)
    X_test_reduced_scaled = scaler.transform(X_test_reduced)
    
    # Пересчитываем p-values для новой модели
    p_values = calculate_p_values(X_train_reduced_scaled, y_train, model)
    
    print(f"Исключен признак: {excluded_feature}")

# Построение финальной модели с оставшимися признаками
X_train_final = X_train[features]
X_test_final = X_test[features]

# Масштабируем данные
X_train_final_scaled = scaler.fit_transform(X_train_final)
X_test_final_scaled = scaler.transform(X_test_final)

model.fit(X_train_final_scaled, y_train)
y_pred = model.predict(X_test_final_scaled)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
print("\nMean Squared Error на тестовой выборке:", mse)

# Выводим таблицу с результатами
final_features = pd.DataFrame({
    "Feature": features,
    "Coefficient": model.coef_
})

print("\nКоэффициенты модели после backward elimination:")
print(final_features)


Исключен признак: chlorides
Исключен признак: total sulfur dioxide
Исключен признак: citric acid

Mean Squared Error на тестовой выборке: 0.5686162151937431

Коэффициенты модели после backward elimination:
               Feature  Coefficient
0        fixed acidity     0.037889
1     volatile acidity    -0.195012
2       residual sugar     0.371090
3  free sulfur dioxide     0.079279
4              density    -0.385756
5                   pH     0.091530
6            sulphates     0.072899
7              alcohol     0.278928
